# Email Service - Send to all consumers

## Download dependencies

In [ ]:
%pip install openai bs4 pandas requests python-docx

## Create Zero Shot prompt
This will be used to guide LLM when classifying text into the 3 political concepts

In [ ]:
text = """
# Identity
You are AI model that classifies text into one of the 3 distinct categories. You will be given text as input. That is what you need to classify into one of the categories. 
The 3 categories are **Immigration**, **Economy**, **Civil**

# Instructions
* Do not answer in a sentence at all. 
* Do not give responses with Markdown formatting, just return a one word answer which corresponds to one of the 3 categories mentioned
* Never answer in a sentence. 
* Respond using EXACTLY one word from the allowed categories.
"""

with open('political_text_classifier.txt', 'w') as f:
    f.write(text)

In [ ]:
text = """
# Identity
You are AI summarizing tool that summarizes political legislation in easy to understand language for the public to better understand the texts.

# Instructions
* Give CLEAR and EASY TO UNDERSTAND explanation of the legislation 
* Only give information about the content of the legislation. Information like the author or date published should be ignored.
* Never add your own comments or thoughts about the text. 
* Give OBJECTIVE information about the content.
* Respond in markdown format with bullet points for easy reading.
"""

with open('political_text_summarizer.txt', 'w') as f:
    f.write(text)

## Load Zero Shot prompt

In [ ]:
with open("political_text_classifier.txt", "r", encoding="utf-8") as file:
    political_text_classifier = file.read()

In [ ]:
with open("political_text_summarizer.txt", "r", encoding="utf-8") as file:
    political_text_summarizer = file.read()

## Define AI functions
This uses OpenAI SDK and GPT 4o as base LLM

### Set up OpenAI client

In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')  # Use environment variable instead
)

### Political text classification

In [ ]:
def classifyText(input_text):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": political_text_classifier
            },
            {"role": "user", "content": input_text}
        ]
    )
    return response.choices[0].message.content.strip().split()[0]

### Summarizer of legislative text

In [ ]:
def summarizeText(input_text: str):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": political_text_summarizer},
            {"role": "user", "content": input_text}
        ]
    )
    return response.choices[0].message.content.strip()

## Build webscraper to get NYC legislation
BeautifulSoup will be used to access HTML of NYC Legistar website. We will save the scraped data in a Pandas dataframe for further processing

### Scrape council committee meetings

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

request_url = "https://legistar.council.nyc.gov/Calendar.aspx?Mode=This+Week"
web_html = requests.get(request_url).text
soup = BeautifulSoup(web_html, "html.parser")
table = soup.find('table', id='ctl00_ContentPlaceHolder1_gridCalendar_ctl00')

council_meetings = []

# Skip the header row which is index 0
# Only get the 2 most recent meetings
for tr in table.find_all('tr')[1:]:  
    cells = tr.find_all('td')

    committee = cells[0].get_text(strip=True)
    date = cells[1].get_text(strip=True)
    meeting_time = cells[3].get_text(strip=True)
    
    if meeting_time == "Deferred":
        continue
    
    # Get the agenda link (from the 7th column)
    meeting_detail = cells[6].find('a')
    meeting_detail_aspx = meeting_detail['href']    
    
    if len(council_meetings) < 2:
        council_meetings.append({
            'Date': date,
            'Committee': committee,
            'Meeting Details': meeting_detail_aspx
        })
    else:
        break

### View council meetings in a formatted manner

In [ ]:
df = pd.DataFrame(council_meetings)
df

### Scrape Meeting Details + Process through AI
- Iterate through each meeting detail and iterate through each "Introduction" legislative text
- Run the text through the 2 AI helper functions

In [ ]:
categories = {
    "Immigration": [],
    "Economy": [],
    "Civil": []
}

In [ ]:
from io import BytesIO
from docx import Document

for meeting in council_meetings:
    meeting_details_url = f"https://legistar.council.nyc.gov/{meeting['Meeting Details']}"
    meeting_details_html = requests.get(meeting_details_url).text
    soup = BeautifulSoup(meeting_details_html, "html.parser")  
    table = soup.find('table', id='ctl00_ContentPlaceHolder1_gridMain_ctl00')
    legislation_file = []
    
    for tr in table.find_all('tr')[1:]:  # Skip header row
        cells = tr.find_all('td')
        file_type = cells[6].get_text(strip=True)
        if file_type != "Introduction":
            continue
        
        file_locator = cells[0].find('a')['href']
        if len(legislation_file) > 2:
            break
        
        legislation_file.append(file_locator)
    
    for file_locator in legislation_file:
        response = requests.get(f"https://legistar.council.nyc.gov/{file_locator}").text
        soup = BeautifulSoup(response, "html.parser")

        span_body = soup.find('span', id="ctl00_ContentPlaceHolder1_lblAttachments2")
        legislation_pdf_link = span_body.find_all('a')[2]['href']
        name = soup.find('span', id="ctl00_ContentPlaceHolder1_lblName2").get_text(strip=True)
        status = soup.find('span', id="ctl00_ContentPlaceHolder1_lblStatus2").get_text(strip=True)

        fetched_document = requests.get(f"https://legistar.council.nyc.gov/{legislation_pdf_link}")
        doc = Document(BytesIO(fetched_document.content))    
        legislation_text = '\n'.join([para.text for para in doc.paragraphs])

        category = classifyText(legislation_text)
        summarized_category = summarizeText(legislation_text)

        categories[category].append({
            "Name": name,
            "Status": status,
            "Summarized": summarized_category
        })

## Email to subscribers

In [ ]:
# TODO: Implement email functionality